# Creamos bucles para el entrenamiento sistematico  
Esto incluye la carga de datos una vez (o varias si no nos cabe en memoria). Y luego el entrenamiento recursivo de modelos con diferentes parámetros. 

Tenemos que automatizar y hacer un buen control de errores en: 

    - Carga de datos.  
        - La carga y el creado de barches etc.  
    - Entrenamiento de modelos.  
    - Sacar las metricas oportunas.  

### Primera parte: Crear bucle carga de datos 

Vamos a tener en cuenta:  
1. Que los datos se carge en memoria todos si caben.  
2. Crear diferentes labels, para poder probar las diferentes opciones de agrupación.
    - Lo hacemos para probar cual es la mejor agrupación. 
    
**El tema de diferentes labels lo ponemos pendiente, por ahora tiramos con las que sabemos que funcionan, que son:**  
- Gamma  
- Electron  
- Hadrones

In [1]:
import sys
sys.path.append('../src/CTA-data-analisis-library/')

In [2]:
#cargamos librerias 
import os 
import subprocess
from datetime import datetime
import numpy as np 
import glob
import matplotlib.pyplot as plt
import tensorflow as tf 
import psutil
import re
import random
import shutil
import pickle

#propias
import unzipdata_and_first_treatments as manipulate
import loaddata4use
import model_creation_functions as models

### Primero vamos a experimentar con el tema de la memoria, para poder entrenar al máximo sin que se quede colgado

Tenemos muy poca ram disponible, por ello, vamos a optar por una alternativa de entrenamiento por fases, intentando que el modelo vea todos los datos posibles, pero que no olvide o se vea sesgado por subconjuntos de datos que se desvíen mucho de la distribución general de estos.

In [3]:
#enviroment variables
npy_final_dir="../datos/elementos_npy"
base_dir_elementos="../datos/elementos"
elements=['gamma', 'electron', 'proton', 'helium', 'iron', 'nitrogen', 'silicon']

Buenos, podemos ver como los archivos de gamma son mucho más pesados, 4 veces o más. 

La idea va a ser hacer un boostrap (no se si es exactamente eso, pero creo que sí). En el que vayamos cogiendo de un conjunto grande de train, vamos a ir escogiendo aleatoriamente, y finalmente, tras muchas cargas entrenamientos y tal, en teoría el modelo las habra visto todas en un orden aleatorio, repitiendo etc.

**Por el momento no voy a hacer un tratamiento intensivo del desbalanceo.**  
**Voy a separar un 10% de las npy files en otro directorio para que podamos coger aleatoriamente del directorio npy_base**

```{bash}
[arturoSF@invidere datos]$ ls elementos_npy_test/*/*.npy | wc -l  
1456  
[arturoSF@invidere datos]$ ls elementos_npy/*/*.npy | wc -l  
6225  
```

### Ciclo de carga de datos 

La idea es cargar x datos en memoria y entrenar, luego cargar otra vez aleatoriamente x datos y sigue.  
Pero la duda es cuantos cargar, porque solo tenemos 5 Gb disponibles y esto aun sin contar con que tenemos el modelo en memoria.


#### Troubleshoting main_list
Necesitamos crear la main_list pero a partir de lo que tenemos disponible, entonces, lo que vamos a hacer es una funcion que escoja segun lo disponible y punto, lo único que necesitamos es extraer las runs disponibles para cada elemento.

In [4]:
def runs_disponibles(npy_dir,elements):
    lista=[]
    for i in elements:
        npy_element_dir=os.path.join(npy_dir,"npy_"+i)
        runs=[int(re.search("run_([0-9]{3})_0\.npy",i).group(1)) for i in os.listdir(npy_element_dir) if re.search("run_([0-9]{3})_0\.npy",i)]
        lista.append(runs)
    return lista

In [5]:

res=[np.array(i) for i in runs_disponibles(npy_final_dir,elements)]
total={}
for i in range(1,180):
    aux=[]
    for j in res:
        aux.append(np.sum(j==i))
        
    total[str(i)]=aux
      
for i in total:
    if ((1 in total[i]) | (2 in total[i]) | (3 in total[i])):
        print("Para la run:", i," hay problemas. ", total[i])

Para la run: 2  hay problemas.  [2, 0, 4, 0, 4, 4, 4]
Para la run: 3  hay problemas.  [3, 4, 4, 4, 4, 4, 4]


Ahora ya tenemos todas las opciones de runs a correr, solo vamos a quitar nosotros la 2 y la 3 de gamma a mano, que dan problemas, y el estos se elegiran de forma aleatoria.

In [6]:
chose_runs=runs_disponibles(npy_final_dir,elements)
chose_runs[0].remove(2)
chose_runs[0].remove(2)
chose_runs[0].remove(3)
chose_runs[0].remove(3)
chose_runs[0].remove(3)
#rapido y mal

In [7]:
def new_create_main_list_runs(number_runs_per_element,posibles_runs):
    #esto es aleatorio por defecto, porque es lo unico que necesito por ahora    
    final=[]
    for ind,lista_runs_element in enumerate(chose_runs):
        final.append(random.sample(lista_runs_element,number_runs_per_element[ind]))
    return final

In [9]:
#prueba de carga de datos 

#en lugar de usar esta carga de datos
#list_runs=loaddata4use.create_main_list_runs(num_events=2,init_events=4)

#usamos esta otra
list_runs=new_create_main_list_runs([6,9,9,9,9,9,9],chose_runs)
list_runs

[[44, 12, 34, 6, 6, 13],
 [153, 87, 142, 30, 73, 3, 113, 100, 103],
 [13, 5, 10, 129, 56, 157, 91, 60, 47],
 [94, 100, 43, 42, 58, 63, 116, 4, 50],
 [167, 119, 81, 107, 37, 94, 98, 60, 2],
 [42, 108, 85, 146, 157, 58, 84, 40, 17],
 [106, 34, 108, 13, 21, 49, 114, 35, 138]]

In [ ]:
x_train_list,x_test_list,y_train_list,y_test_list=loaddata4use.load_dataset_completo(npy_final_dir,labels_asign=[0,1,2,2,2,2,2],elements=elements,
                                                                                     main_list_runs=list_runs,pre_name_folders="npy_",telescopes=[1,2,3,4],
                                                                                     test_size=0.05,same_quant="same",verbose=True,fill=True,categorical=True)

Load of names and common events
_______________
0 ;  Element:  gamma  , Runs:  [44, 12, 34, 6, 6, 13]  Shape of common events (tels,common events):  (4, 1086)
1 ;  Element:  gamma  , Runs:  [44, 12, 34, 6, 6, 13]  Shape of common events (tels,common events):  (4, 1147)
2 ;  Element:  gamma  , Runs:  [44, 12, 34, 6, 6, 13]  Shape of common events (tels,common events):  (4, 1209)
3 ;  Element:  gamma  , Runs:  [44, 12, 34, 6, 6, 13]  Shape of common events (tels,common events):  (4, 1154)
4 ;  Element:  gamma  , Runs:  [44, 12, 34, 6, 6, 13]  Shape of common events (tels,common events):  (4, 1154)
5 ;  Element:  gamma  , Runs:  [44, 12, 34, 6, 6, 13]  Shape of common events (tels,common events):  (4, 1147)
0 ;  Element:  electron  , Runs:  [153, 87, 142, 30, 73, 3, 113, 100, 103]  Shape of common events (tels,common events):  (4, 232)
1 ;  Element:  electron  , Runs:  [153, 87, 142, 30, 73, 3, 113, 100, 103]  Shape of common events (tels,common events):  (4, 233)
2 ;  Element:  electron 

In [ ]:
#tenemos que hacer un ligero cambio porque se estan cargando con los ejes cambiados
def cambiar_ejes_lista(lista):
    for i,j in enumerate(lista):
        lista[i]=np.swapaxes(j,1,2)
    return lista

x_train_list=cambiar_ejes_lista(x_train_list)
x_test_list=cambiar_ejes_lista(x_test_list)


Ya hemos reproducido la carga de datos de una manera que además nos va a permitir hacer un boostrap para entrenar con recursos limitados.  

Ahora tenemos que ver una forma optima de entrenar los modelos de forma automática.

De este notebook voy a reutilizar las funcions que sacan los runs y que nos dan las main_listas, y con ello, en un script de pipeline, lo llamaré despues con un script de automatizaciones. 

In [ ]:
opciones_filtros=[
    [[12,16],[32,64],[64,12]],
    [[12,16,32],[64,128],[128,64,32]],
    [[12,16],[32,64],[64,12]],
    [[12,16],[32,64],[64,12]],
    [[12,16],[32,64],[64,12]]
]

In [ ]:
modelo=models.model_multi_tel(classes=3,filtros=opciones_filtros[1],last_dense=[20,10])
modelo.summary()

In [ ]:
modelo.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["acc","AUC","mean_squared_error"])

In [ ]:
hist=modelo.fit(x=x_train_list,y=y_train_list,epochs=15, validation_data=(x_test_list,y_test_list),batch_size=64)
modelo.save("../modelos/modelo_1.h5")
with open("../modelos/performances/history_modelo_1.pickle","wb") as pick:
    pickle.dump(hist,pick)

In [ ]:
%%javascript
IPython.notebook.save_notebook()